In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR
import holodeck.anisotropy as anis

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py

In [ ]:
nn=346
sspath = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/'
hdfname = sspath+'sam_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][nn]
hc_bg = ssfile['hc_bg'][nn]
fobs = ssfile['fobs'][:]
# dfobs = ssfile['dfobs'][:]
ssfile.close()

shape=hc_ss.shape
nfreqs = shape[-3]
nreals = shape[-2]
nloudest = shape[-1]
print('nfreqs=%d, nreals=%d, nloudest=%d' % (nfreqs, nreals, nloudest))

# 1 hc


## Place single sources at random pixels

In [ ]:
def convert_pix_ss(pix_ss1, nside1, nside2):
    theta, phi = hp.pix2ang(nside1, pix_ss1)
    print(f"{theta.shape=}, {phi.shape=}")
    pix_ss2 = hp.ang2pix(nside2, theta, phi).reshape(pix_ss1.shape)
    return pix_ss2

# 4 hc / dOmega


## 4.0 Load NPZ

In [ ]:
save_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/healpix/varying_nside_testnpz'
hcfile = np.load(save_loc+'/hcdOm_A.npz')

# pix_ss08=pix_ss08, pix_ss16=pix_ss16, pix_ss32=pix_ss32, pix_ss64=pix_ss64, pix_ss128=pix_ss128,
# moll_ss08=moll_ss08, moll_ss16=moll_ss16, moll_ss32=moll_ss32, moll_ss64=moll_ss64, moll_ss128=moll_ss128,
# moll_bg08=moll_bg08, moll_bg16=moll_bg16, moll_bg32=moll_bg32, moll_bg64=moll_bg64, moll_bg128=moll_bg128,
# moll_tt08=moll_tt08, moll_tt16=moll_tt16, moll_tt32=moll_tt32, moll_tt64=moll_tt64, moll_tt128=moll_tt128,
# pix_ss08 = hcfile['pix_ss08']
# pix_ss16 = hcfile['pix_ss16']
# pix_ss32 = hcfile['pix_ss32']
# pix_ss64 = hcfile['pix_ss64']
# # pix_ss128 = hcfile['pix_ss128']
CldOm_arrs = hcfile['CldOm_arrs']
# nsides=hcfile['nsides']

hcfile.close()

# lmax=6

## Maps

In [ ]:
# spread background evenly across pixels

def make_hcdOm_bg_map(hc_bg, nside):
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    moll_bg = (np.ones((hc_bg.shape + (npix,))) 
            * hc_bg[:,:,np.newaxis]/np.sqrt(npix)/area) # (frequency, realization, pixel)
    return moll_bg


In [ ]:

nsides=np.array([8,16,32, 64]) #, 128])
molldOm_bg08 = make_hcdOm_bg_map(hc_bg, nside=8)
molldOm_bg16 = make_hcdOm_bg_map(hc_bg, nside=16)
molldOm_bg32 = make_hcdOm_bg_map(hc_bg, nside=32)
molldOm_bg64 = make_hcdOm_bg_map(hc_bg, nside=64)
# molldOm_bg128 = make_hcdOm_bg_map(hc_bg, nside=128)
print(molldOm_bg08.shape)


In [ ]:

# place single sources at random pixels, for one map

def make_hcdOm_ss_map(hc_ss, nside, pix_ss=None):
    print(hc_ss.shape)
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    if pix_ss is None:
        pix_ss = np.random.randint(0, npix-1, size=hc_ss.size).reshape(hc_ss.shape)
    print(pix_ss.shape)
    moll_ss = np.zeros((len(hc_ss), len(hc_ss[0]), npix,))
    print(f"{npix=}, {moll_ss.shape=}")
    for ff in range(hc_ss.shape[-3]):
        for rr in range(hc_ss.shape[-2]):
            for ll in range(hc_ss.shape[-1]):
                moll_ss[ff,rr,pix_ss[ff,rr,ll]] = (np.sqrt(moll_ss[ff,rr,pix_ss[ff,rr,ll]]**2 
                                                   + hc_ss[ff,rr,ll]**2) /area)
    return moll_ss, pix_ss


In [ ]:

molldOm_ss08, pix_ss08 = make_hcdOm_ss_map(hc_ss, 8)
pix_ss16 = convert_pix_ss(pix_ss08, 8, 16)
pix_ss32 = convert_pix_ss(pix_ss08, 8, 32)
pix_ss64 = convert_pix_ss(pix_ss08, 8, 64)

molldOm_ss16, temp = make_hcdOm_ss_map(hc_ss, 16, pix_ss16)
molldOm_ss32, temp = make_hcdOm_ss_map(hc_ss, 32, pix_ss32)
molldOm_ss64, temp = make_hcdOm_ss_map(hc_ss, 64, pix_ss64)


In [ ]:
molldOm_tt08 = np.sqrt(molldOm_ss08**2 + molldOm_bg08**2)
molldOm_tt16 = np.sqrt(molldOm_ss16**2 + molldOm_bg16**2)
molldOm_tt32 = np.sqrt(molldOm_ss32**2 + molldOm_bg32**2)
molldOm_tt64 = np.sqrt(molldOm_ss64**2 + molldOm_bg64**2)


In [ ]:

rr=0
for ff in range(0,5):
    fig, axs = plot.figax(ncols=3, figsize=(15,5))
    plt.axes(axs[0])
    hp.mollview(molldOm_tt08[ff,rr], unit='$h_c/d\Omega$',
                title='f=%.2f yr$^{-1}$, nside=%d' % (fobs[ff]*YR,  8), hold=True )
    plt.axes(axs[1])
    hp.mollview(molldOm_tt16[ff,rr], unit='$h_c/d\Omega$',
                title='f=%.2f yr$^{-1}$, nside=%d' % (fobs[ff]*YR, 16), hold=True )
    plt.axes(axs[2])
    hp.mollview(molldOm_tt32[ff,rr], unit='$h_c/d\Omega$',
                title='f=%.2f yr$^{-1}$, nside=%d' % (fobs[ff]*YR, 32), hold=True )
    fig.tight_layout()


## 4.2 sph harm

In [ ]:
lmax=6
CldOm_08 = anis.sph_harm_from_map(molldOm_tt08, lmax=6)
CldOm_16 = anis.sph_harm_from_map(molldOm_tt16, lmax=6)
CldOm_32 = anis.sph_harm_from_map(molldOm_tt32, lmax=6)
CldOm_64 = anis.sph_harm_from_map(molldOm_tt64, lmax=6)

In [ ]:
CldOm_arrs = np.array([CldOm_08, CldOm_16, CldOm_32, CldOm_64]) #, CldOm_128])
print(CldOm_arrs.shape)

In [ ]:
rr = 0
ll=1

xx = np.arange(lmax+1)
colors = cm.rainbow(np.linspace(0,1,len(CldOm_arrs)))

for ff in np.array([1,5,10]):
    yy = CldOm_arrs
    fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
    for ii, nside in enumerate(nsides):
        ax.plot(xx, yy[ii,ff,rr,:], color=colors[ii], label=('$\ell=%d$, $\ell_\mathrm{max}=%d,$ nside=%d' % (ll, lmax, nside)))
    ax.legend()
    ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c/d\Omega$' % (nn, rr, fobs[ff]*YR))
    fig.tight_layout()

In [ ]:
xx = fobs*YR
colors = cm.rainbow(np.linspace(0,1,len(CldOm_arrs)))

for ll in range(1,7):
    yy = CldOm_arrs[...,ll]/CldOm_arrs[...,0]
    fig, ax = plot.figax(xlabel='$f$ yr$^{-1}$', ylabel='$C_1/C_0$')
    for ii, nside in enumerate(nsides):
        ax.plot(xx, np.median(yy[ii], axis=-1), color=colors[ii], label=('$\ell=%d$, $\ell_\mathrm{max}=%d,$ nside=%d' % (ll, lmax, nside)))
        for pp in [50,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[ii], percs, axis=-1), alpha=0.25, color=colors[ii])

    ax.legend()
    ax.set_title('Sample %d, Realization %d, $h_c/d\Omega$' % (nn, rr))
    fig.tight_layout()

## 4.3 Save npz

In [ ]:
# save_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/healpix/varying_nside_testnpz'
# np.savez(save_loc+'/hcdOm_A.npz', 
#          pix_ss08=pix_ss08, pix_ss16=pix_ss16, pix_ss32=pix_ss32, pix_ss64=pix_ss64, #pix_ss128=pix_ss128,
#          molldOm_ss08=molldOm_ss08, molldOm_ss16=molldOm_ss16, molldOm_ss32=molldOm_ss32, molldOm_ss64=molldOm_ss64, #moll2dOm_ss128=moll2dOm_ss128,
#          molldOm_bg08=molldOm_bg08, molldOm_bg16=molldOm_bg16, molldOm_bg32=molldOm_bg32, molldOm_bg64=molldOm_bg64,# moll_bg128=moll_bg128,
#          molldOm_tt08=molldOm_tt08, molldOm_tt16=molldOm_tt16, molldOm_tt32=molldOm_tt32, molldOm_tt64=molldOm_tt64, #moll_tt128=moll_tt128,
#          CldOm_arrs=CldOm_arrs, nsides=nsides)
#         #  Cl_08=Cl_08, Cl_16=Cl_16, Cl_32=Cl_32, Cl_64=Cl_64, Cl_128=Cl_128)

# Simple Function

$$ a_{\ell m} = \int d\Omega_n \Delta T (\vec{n}) Y^*_{\ell m} (\vec{n}) $$
$$ C_{\ell m} = \frac{1}{2\ell + 1} \sum_{m=-\ell}^\ell |a_{\ell m}|^2 $$

$$ C_0 = (1) \int d\Omega \cdot \frac{h_c^2}{d\Omega} \cdot \mathrm {constant} $$

In [ ]:
def solid_angle(nside):
    area = hp.nside2pixarea(nside, degrees=False) # 
    return area

def C0_integral(moll2_ss, nside, Y00star=1):
    sum=0
    dOm = solid_angle(nside)
    for ii, hc2 in enumerate(moll2_ss):
        sum += Y00star * hc2 * dOm
    return sum

ff = 1
rr = 0
dOmega =solid_angle(nside=8)
print(dOmega)
C0ss_08 = C0_integral(moll2dOm_ss08[ff,rr], 8)
C0ss_16 = C0_integral(moll2dOm_ss16[ff,rr], 16)
C0ss_32 = C0_integral(moll2dOm_ss32[ff,rr], 32)
C0ss_64 = C0_integral(moll2dOm_ss64[ff,rr], 64)
C0ss_128 = C0_integral(moll2dOm_ss128[ff,rr], 125)


C0tt_08 = C0_integral(moll2dOm_tt08[ff,rr], 8)
C0tt_16 = C0_integral(moll2dOm_tt16[ff,rr], 16)
C0tt_32 = C0_integral(moll2dOm_tt32[ff,rr], 32)
C0tt_64 = C0_integral(moll2dOm_tt64[ff,rr], 64)
C0tt_128 = C0_integral(moll2dOm_tt128[ff,rr], 125)

In [ ]:
print(moll_ss08.shape)

In [ ]:
nsides = np.array([8, 16, 32, 64, 128])
C0ss = np.array([C0ss_08, C0ss_16, C0ss_32, C0ss_64, C0ss_128])
C0tt = np.array([C0tt_08, C0tt_16, C0tt_32, C0tt_64, C0tt_128])

fig, ax = plot.figax(xlabel='nside', ylabel=r'$\int d\Omega \frac{h_c^2}{d\Omega}$')
ax.plot(nsides, C0ss, marker='o', label='ss only')
ax.plot(nsides, C0tt, marker='o', label='total map')

l/ 2 pi r = theta / 2 pi 
l = theta r
l^2 = theta^2 r^2 = area
omega = area / r^2 = theta ^2 r^2 / r^2 = theta^2

In [ ]:
rat = 0.1
print(rat, rat**2)
rat = 10
print(rat, rat**2)

# Scratch

In [ ]:
nbest=100
split=2
for ss in range(split):
    bestrange = (np.array([ss, (ss+1)])*(nbest)/split).astype(int)
    bestrange[1] = np.min([bestrange[1], nbest])
    print(bestrange)